# 通过开源模型直接在本地转录

In [1]:
!apt update &&  apt install -y ffmpeg

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:2 https://deb.nodesource.com/node_18.x nodistro InRelease                  
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                    
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.cn/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Fetched 119 kB in 2s (57.2 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
31 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [2]:
%pip install openai-whisper
%pip install setuptools-rust

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [4]:
import whisper

model = whisper.load_model("large")
def transcript(clip, prompt, output):
    result = model.transcribe(clip, initial_prompt=prompt)
    with open(output, "w") as f:
        f.write(result['text'])
    print("Transcripted: ", clip)
    
prompt = "这是一段Onboard播客，里面会聊到ChatGPT以及PALM这个大语言模型。这个模型也叫做Pathways Language Model。\n\n"

clip = f"./data/podcast_clip.mp3"
output = f"./data/transcripts/podcast_clip.txt"
transcript(clip, prompt, output)
with open(output, "r") as f:
    transcript = f.read()
    print(transcript)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:65: UserWarning: /root/.cache/whisper/large-v3.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.88G/2.88G [13:18<00:00, 3.87MiB/s]


Transcripted:  ./data/podcast_clip.mp3
欢迎来到Onboard,真实的一线经验,走新的投资思考。我是Monica。我是高宁,我们一起聊聊软件如何改变世界。大家好,欢迎来到Onboard,我是Monica。自从OpenAI发布的ChatGPT掀起了席卷世界的AI热潮,不到三个月就积累了超过一亿的月活用户,超过1300万的日活用户。真的是展现了AI让人惊叹的能力,也让很多人直呼这就是下一个互联网的未来。有不少观众都说,希望我们再做一期AI的讨论,于是这次硬核讨论就来了。这次我们请来了Google Brain的研究员雪芝,她是Google大语言模型PALM,Pathways Language Model的作者之一。要知道,这个模型的参数量是GPT-3的三倍还多。另外还有两位AI产品大牛,一位来自著名的StableModels,StableDiffusion背后的商业公司StabilityAI,另一位来自某硅谷科技大厂,也曾在吴恩达教授的LandingAI中担任产品负责人。此外,莫妮凯还邀请到一位一直关注AI的投资人朋友Bill,当作我的特邀共同主持嘉宾。我们主要讨论几个话题,一方面,从研究的视角,最前沿的研究者在关注什么?现在的技术的天花板和未来大的变量可能会在哪里?从产品和商业的角度,什么是一个好的AI产品?整个状态可能随着技术有怎样的演变?更重要的,我们又能从上一波AI的创业热潮中学到什么?最后,莫妮凯和Bill还会从投资人的视角做一个回顾、总结和畅想。这里还有一个小的update,在本集发布的时候,Google也对爆发式增长的ChatGPT做出了回应,正在测试一个基于Lambda模型的聊天机器人ApprenticeBot,正式发布,我们都会有什么惊喜,我们都拭目以待。AI无理是未来几年最令人兴奋的变量之一,莫妮凯也希望未来能邀请到更多一线从业者从不同角度讨论这个话题,不论是想要做创业、研究、产品还是投资的同学,希望这些对话对于大家了解这些技术演进、商业的可能,甚至未来对于我们每个人、每个社会意味着什么,都能引发一些思考,提供一些启发。这些对话,这次的讨论有些技术硬核,需要各位对生存式AI大模型都有一些基础了解,讨论中涉及到的论文和重要概念,也会总结在本集的简介中,供大家复习参考。几位嘉宾在北美工作生活多年,夹杂英文在所难免,也请

# 结合ChatGPT做内容小结

In [5]:
from langchain_openai import ChatOpenAI
from langchain.text_splitter import SpacyTextSplitter
from llama_index.core import GPTListIndex, ServiceContext, SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser

llm = ChatOpenAI(temperature=0,model_name="gpt-3.5-turbo", max_tokens=1024)
text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
parser = SimpleNodeParser(chunking_tokenizer_fn=text_splitter.split_text)
documents = SimpleDirectoryReader('./data/transcripts').load_data()
nodes = parser.get_nodes_from_documents(documents)

service_context  = ServiceContext.from_defaults(llm=llm)
index = GPTListIndex(nodes=nodes, service_context=service_context)
query_engine = index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("请你用中文总结一下我们的播客内容:")
print(response)

/tmp/ipykernel_93/770114377.py:12: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context  = ServiceContext.from_defaults(llm=llm)
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


播客内容主要围绕AI技术的发展和应用展开讨论，涉及到了ChatGPT等大型AI模型的影响和潜力，以及从研究、产品、商业和投资等多个角度对AI的探讨。嘉宾们分享了他们的经验和见解，展望了AI技术的未来发展，并提供了对AI技术演进和商业应用的启发和思考。
